# HW3P2: LibriSpeech ASR With Phonetic Transcription

## Overview

This repository contains the implementation for **Homework 3 Part 2** of the Automatic Speech Recognition (ASR) course. The task focuses on building an ASR system using the LibriSpeech dataset with phonetic transcription. It involves designing a deep learning model with a focus on feature extraction, sequence modeling, and decoding phonetic transcriptions.

## Features

### Model Architecture

1. **Encoder**:
   - **Convolutional Layers**:
     - Input size: Matches the feature size of the audio spectrogram.
     - Three convolutional layers with the following dimensions:
       - Layer 1: 1D Conv with input size `input_size`, output size 256.
       - Layer 2: 1D Conv with input size 256, output size 256.
       - Layer 3: 1D Conv with input size 256, output size 512.
     - Activation: GELU.
     - Normalization: BatchNorm1d applied after each convolutional layer.
   - **pBLSTM**:
     - Number of layers: 4.
     - Hidden size: Configurable; default set to 128 in the script.
     - Dropout: Locked dropout applied with probabilities 0.4 and 0.3 between layers.

2. **Decoder**:
   - **MLP Structure**:
     - Input size: Double the output size of the encoder's last pBLSTM layer.
     - Layer 1: Linear with output size 2048, followed by GELU and dropout (0.2).
     - Layer 2: Linear with output size 1024, followed by GELU and dropout (0.2).
     - Layer 3: Linear with output size 512, followed by GELU and dropout (0.2).
     - Output layer: Linear with output size equal to the number of phonemes (default 41).
   - Log-Softmax applied to the output for probability normalization.

3. **ASRModel**:
   - Combines the encoder and decoder into a single module.
   - Outputs phoneme probabilities for each timestep.

### Key Parameters

- **Encoder**:
  - Input size: Configurable based on the feature extraction process.
  - Hidden size for pBLSTM: 128.
  - Number of pBLSTM layers: 4.
- **Decoder**:
  - MLP hidden layer sizes: [2048, 1024, 512].
  - Output size: 41 phonemes.
- **General Settings**:
  - Batch size: 128.
  - Learning rate: \( 2 \times 10^{-3} \).
  - Dropout rates: 0.2 in MLP layers, 0.4 and 0.3 in pBLSTM layers.
  - Beam width: 5 (used during decoding).

## Additional Details

### Data Processing

The input consists of spectrogram features processed into sequences suitable for the encoder. The pBLSTM layers progressively reduce the sequence length while increasing feature dimensionality, making the system efficient for longer input sequences.

### Decoding

The beam search decoder leverages the output probabilities to generate phonetic sequences, and the Levenshtein distance function evaluates alignment between predicted and true sequences.

# Installs

In [1]:
!pip install torchinfo
!pip install wandb --quiet
!pip install python-Levenshtein -q
!pip install flashlight-text -q

# Imports

In [2]:
import torch
import numpy as np
import torch.nn as nn
from torchinfo import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import torchaudio.transforms as tat
import pandas as pd
from tqdm import tqdm
import os
import Levenshtein
from torchaudio.models.decoder import ctc_decoder
from torchaudio.models.decoder import cuda_ctc_decoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


# Dataset and Dataloader

In [3]:
# ARPABET PHONEME MAPPING
# DO NOT CHANGE

CMUdict_ARPAbet = {
    "" : " ",
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@",
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W",
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R",
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w",
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y",
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D",
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O",
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"
}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict[:-2]
LABELS = ARPAbet[:-2]

### Train Data

In [4]:
class AudioDataset(Dataset):
    def __init__(self, mfcc_dir, transcript_dir=None, epsilon=1e-8):
        self.epsilon = epsilon
        self.phoneme_to_id = {phoneme: i for i, phoneme in enumerate(PHONEMES)}
        self.mfcc_data = []
        mfcc_files = sorted([
            f for f in os.listdir(mfcc_dir) 
            if f.endswith('.npy')
        ])
        self.mfcc_files = [os.path.join(mfcc_dir, f) for f in mfcc_files]
        print("Loading and normalizing MFCC data...")
        for mfcc_file in self.mfcc_files:
            mfcc = np.load(mfcc_file)
            mfcc = self.cmn(mfcc)
            mfcc_tensor = torch.tensor(mfcc, dtype=torch.float32)
            self.mfcc_data.append(mfcc_tensor)
        print(f"Loaded {len(self.mfcc_data)} MFCC samples.")

        if transcript_dir:
            transcript_files = sorted([
                f for f in os.listdir(transcript_dir) 
                if f.endswith('.npy')
            ])
            self.transcript_files = [os.path.join(transcript_dir, f) for f in transcript_files]
            assert len(self.transcript_files) == len(self.mfcc_files)
            self.transcript_data = []
            print("Loading and processing transcript data...")
            for transcript_file in self.transcript_files:
                transcript = np.load(transcript_file)
                transcript_ids = [
                    self.phoneme_to_id.get(phoneme, -1) 
                    for phoneme in transcript
                ]
                transcript_ids = [t for t in transcript_ids if t != -1]
                transcript_tensor = torch.tensor(transcript_ids, dtype=torch.long)
                self.transcript_data.append(transcript_tensor)
            print(f"Loaded {len(self.transcript_data)} transcript samples.")
        else:
            self.transcript_data = None
            print("No transcript data provided.")
    
    def cmn(self, mfcc):
        mean = np.mean(mfcc, axis=0)
        std = np.std(mfcc, axis=0) + self.epsilon
        normalized_mfcc = (mfcc - mean) / std
        return normalized_mfcc
    
    def __len__(self):
        return len(self.mfcc_data)
    
    def __getitem__(self, idx):
        mfcc = self.mfcc_data[idx]
        if self.transcript_data:
            transcript = self.transcript_data[idx]
        else:
            transcript = None
        return mfcc, transcript
    
    def collate_fn(self, batch):
        mfccs, transcripts = zip(*batch)
        mfcc_lengths = torch.tensor([mfcc.shape[0] for mfcc in mfccs], dtype=torch.long)
        if transcripts[0] is not None:
            transcript_lengths = torch.tensor([transcript.shape[0] for transcript in transcripts], dtype=torch.long)
            transcripts_padded = pad_sequence(transcripts, batch_first=True, padding_value=-1)  # Assuming -1 is used for padding
        else:
            transcript_lengths = None
            transcripts_padded = None
        mfccs_padded = pad_sequence(mfccs, batch_first=True)
        return mfccs_padded, transcripts_padded, mfcc_lengths, transcript_lengths

### Config - Hyperparameters

In [5]:
config = {
    "beam_width": 5,
    "lr": 2e-3,  
    "epochs": 80,  
    "batch_size": 128,  
    "dropout": 0.2,
    "embed_size": 128, 
}

last_epoch_completed = 0
start = last_epoch_completed
end = config["epochs"]
best_lev_dist = float("inf")

epoch_model_path = "/kaggle/working/epoch_checkpoint.pth"
best_model_path = "/kaggle/working/best_model.pth"
csv_path = "/kaggle/working/submission.csv"
root_dir = "/kaggle/input/11-785-hw3p2-f24/11785-f24-hw3p2/"

class PermuteBlock(torch.nn.Module):
    def forward(self, x):
        return x.transpose(1, 2)
    
transforms = nn.Sequential(
    tat.FrequencyMasking(freq_mask_param=5),
    tat.TimeMasking(time_mask_param=100)
 )

### Data loaders

In [6]:
# get me RAMMM!!!!
import gc
gc.collect()

0

In [7]:
train_data = AudioDataset("/kaggle/input/11-785-hw3p2-f24/11785-f24-hw3p2/train-clean-100/mfcc", "/kaggle/input/11-785-hw3p2-f24/11785-f24-hw3p2/train-clean-100/transcript")
train_loader = DataLoader(train_data, batch_size=config["batch_size"], collate_fn=train_data.collate_fn, shuffle=True, num_workers=0, pin_memory=True)
val_data = AudioDataset("/kaggle/input/11-785-hw3p2-f24/11785-f24-hw3p2/dev-clean/mfcc", "/kaggle/input/11-785-hw3p2-f24/11785-f24-hw3p2/dev-clean/transcript")
val_loader = DataLoader(val_data, batch_size=config["batch_size"], collate_fn=val_data.collate_fn, shuffle=False, num_workers=0, pin_memory=True)
test_data = AudioDataset("/kaggle/input/11-785-hw3p2-f24/11785-f24-hw3p2/test-clean/mfcc")
test_loader = DataLoader(test_data, batch_size=config["batch_size"], collate_fn=test_data.collate_fn, shuffle=False)

gc.collect()

print("Batch size: ", config["batch_size"])
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Loading and normalizing MFCC data...
Loaded 28539 MFCC samples.
Loading and processing transcript data...
Loaded 28539 transcript samples.
Loading and normalizing MFCC data...
Loaded 2703 MFCC samples.
Loading and processing transcript data...
Loaded 2703 transcript samples.
Loading and normalizing MFCC data...
Loaded 2620 MFCC samples.
No transcript data provided.
Batch size:  128
Train dataset samples = 28539, batches = 223
Val dataset samples = 2703, batches = 22
Test dataset samples = 2620, batches = 21


In [8]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break

torch.Size([128, 1672, 28]) torch.Size([128, 207]) torch.Size([128]) torch.Size([128])


# NETWORK

## ASR Network

In [9]:
class LockedDropout(nn.Module):
    def __init__(self, drop_prob):
        super(LockedDropout, self).__init__()
        self.prob = drop_prob
    def forward(self, x):
        if not self.training or not self.prob:
            return x
        x, x_lens = pad_packed_sequence(x, batch_first = True)
        m = x.new_empty(x.size(0), 1, x.size(2),requires_grad=False).bernoulli_(1 - self.prob)
        mask = m / (1 - self.prob)
        mask = mask.expand_as(x)
        out = x * mask
        out = pack_padded_sequence(out,x_lens, batch_first = True, enforce_sorted= False)
        return out

### Pyramid Bi-LSTM (pBLSTM)

In [10]:
# Utils for network
torch.cuda.empty_cache()

In [11]:
class pBLSTM(torch.nn.Module):
    
    def __init__(self, input_size, hidden_size):
        super(pBLSTM, self).__init__()
        self.blstm = nn.LSTM(input_size=2*input_size, hidden_size=hidden_size, num_layers=4, bidirectional=True, dropout=0.2, batch_first=True) 

    def forward(self, x_packed):   
        x, lengths = pad_packed_sequence(x_packed, batch_first=True)
        x, x_lens = self.trunc_reshape(x, lengths)
        x = pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)
        x, h = self.blstm(x)
        
        return x

    def trunc_reshape(self, x, x_lens): 
        if x.shape[1] % 2 != 0:
            x = x[:, :-1, :]
        x = x.reshape(x.shape[0], x.shape[1] // 2, x.shape[2] * 2)
        x_lens = x_lens // 2
        return x, x_lens

### Encoder

In [12]:
class Encoder(nn.Module):  
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        expand_dims = [256, 512]  

        self.embed = nn.Sequential(
            PermuteBlock(),  
            nn.Conv1d(in_channels=input_size, out_channels=expand_dims[0], kernel_size=3, stride=1, padding=1),  
            nn.BatchNorm1d(num_features=expand_dims[0]),  
            nn.GELU(),
            # Additional convolutional layers
            nn.Conv1d(in_channels=expand_dims[0], out_channels=expand_dims[0], kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(num_features=expand_dims[0]),
            nn.GELU(),
            nn.Conv1d(in_channels=expand_dims[0], out_channels=expand_dims[1], kernel_size=3, stride=1, padding=1),  
            nn.BatchNorm1d(num_features=expand_dims[1]),  
            PermuteBlock()  
        )

        self.pBLSTMs = nn.Sequential(
            pBLSTM(input_size=expand_dims[1], hidden_size=hidden_size),  
            LockedDropout(0.4),  
            pBLSTM(input_size=2*hidden_size, hidden_size=hidden_size),  
            LockedDropout(0.3),
        )
         
    def forward(self, x, lens):
        x = self.embed(x)  
        lens = lens.clamp(max=x.shape[1]).cpu()  
        
        x = pack_padded_sequence(x, lens, batch_first=True, enforce_sorted=False)  
        x = self.pBLSTMs(x)  
        outputs, lens = pad_packed_sequence(x, batch_first=True)  

        return outputs, lens  

### Decoder

In [13]:
class Decoder(torch.nn.Module):

    def __init__(self, embed_size, output_size= 41):
        super().__init__()

        self.mlp = torch.nn.Sequential(
            PermuteBlock(), 
            torch.nn.BatchNorm1d(embed_size), 
            PermuteBlock(),
            nn.Linear(embed_size, 2048),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(2048, 1024),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 512),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(512, output_size)
        )
        
        self.softmax = torch.nn.LogSoftmax(dim=2)

    def forward(self, encoder_out):
        out = self.mlp(encoder_out)
        out = self.softmax(out)

        return out 

In [14]:
class ASRModel(torch.nn.Module):

    def __init__(self, input_size, embed_size=192, output_size=len(PHONEMES)):
        super().__init__()
        self.encoder = Encoder(input_size, embed_size)  
        self.decoder = Decoder(embed_size*2, output_size)  
    
    def forward(self, x, lengths_x):
        encoder_out, encoder_lens = self.encoder(x, lengths_x)  
        decoder_out = self.decoder(encoder_out)  

        return decoder_out, encoder_lens  

## Initialize ASR Network

In [15]:
model = ASRModel(
    input_size  = 28,
    embed_size  = config["embed_size"],
    output_size = len(PHONEMES)
).to(device)
# print(model)
summary(model, input_data=[x.to(device), lx.to(device)])

Layer (type:depth-idx)                   Output Shape              Param #
ASRModel                                 [128, 418, 41]            --
├─Encoder: 1-1                           [128, 418, 256]           --
│    └─Sequential: 2-1                   [128, 1672, 512]          --
│    │    └─PermuteBlock: 3-1            [128, 28, 1672]           --
│    │    └─Conv1d: 3-2                  [128, 256, 1672]          21,760
│    │    └─BatchNorm1d: 3-3             [128, 256, 1672]          512
│    │    └─GELU: 3-4                    [128, 256, 1672]          --
│    │    └─Conv1d: 3-5                  [128, 256, 1672]          196,864
│    │    └─BatchNorm1d: 3-6             [128, 256, 1672]          512
│    │    └─GELU: 3-7                    [128, 256, 1672]          --
│    │    └─Conv1d: 3-8                  [128, 512, 1672]          393,728
│    │    └─BatchNorm1d: 3-9             [128, 512, 1672]          1,024
│    │    └─PermuteBlock: 3-10           [128, 1672, 512]         

# Training Config
Initialize Loss Criterion, Optimizer, CTC Beam Decoder, Scheduler, Scaler (Mixed-Precision), etc.

In [16]:
criterion = torch.nn.CTCLoss(blank=0, reduction='mean', zero_infinity=False)  
optimizer = torch.optim.AdamW(model.parameters(), lr=config['lr'])  
decoder = cuda_ctc_decoder(tokens= LABELS, beam_size = config['beam_width'])
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config['epochs'], eta_min=1e-6)

# Mixed Precision, if you need it
scaler = torch.amp.GradScaler('cuda')

# Decode Prediction

In [17]:
def decode_prediction(output, output_lens, decoder, batch_size=config["batch_size"]):
    logits_len = output_lens.to('cuda')
    decoded_text = decoder(output, logits_len.to(torch.int32))
    results = []
    for i in range(len(logits_len)):
        item = decoded_text[i][0]
        words = item.words
        words_string = ''.join(words)
        results.append(words_string)
    
    return results

def calculate_levenshtein(output, label, output_lens, label_lens, decoder, PHONEME_MAP= LABELS):
    
    dist = 0
    batch_size = label.shape[0]
    
    logits_len = output_lens.to('cuda')
    decoded_text = decoder(output, logits_len.to(torch.int32))
    
    for i in range(batch_size):
        pred_string = ''.join([c for c in decoded_text[i][0].words])
        label_string = ''.join([PHONEME_MAP[n] for n in label[i][:label_lens[i]]])
        dist += Levenshtein.distance(pred_string, label_string)

    dist /= batch_size
    
    return dist

# Test Implementation

In [18]:
# test code to check shapes
model.eval()
for data in train_loader:
    x, y, lx, ly = data
    x, y = x.to(device), y.to(device)
    with torch.inference_mode():
        h, lh = model(x, lx)
        h = torch.permute(h, (1, 0, 2))
        loss = criterion(h, y, lh, ly)
    vdist = calculate_levenshtein(torch.permute(h, (1, 0, 2)), y, lh, ly, decoder)
    print("{:.04f}".format(float(vdist)))
    str = decode_prediction(x, lx, decoder)
    print(str[0])
    break

124.6797
@A@v@u@y@un@nuvt@Ay@v-rryftie@f@kUi-f-GkGkGktGtrAvRvuAGrAeUumym-m-muiv@GGrGeCiGia-AamkiWG@y@Aniu-u-uRrZtkRweuekekeAetrzZyi-ti-@CkZi@RA@C-ivyAfbGtvRafae-a-muRwuZafWeryRyv@W-i-iaRWRWrUrWefe@uyAzeZnkz@Azknze@ACZCAfRf-REku-tGefie-m-uzuAumkvRrArCRCrkrRruCZue@ZuZrCRCuRyuAyvAZyuvuyvRuZnuyuez@zeZuRuruZuZGfGv-eAvzmui-v-AekGkZiAymvryruAzRzuR


# WandB

You will need to fetch your api key from wandb.ai

In [19]:
import wandb
wandb.login(key="98642f33baa53793f08e5f32f1d09da8c7c6e80b")

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [20]:
run = wandb.init(
        reinit=True,
        project="HW3P2",
        config=config
    )

wandb: Currently logged in as: seitomoyi (seitomoyi-carnegie-mellon-university). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.3
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241104_173225-zv2rmfw7
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run scarlet-energy-21
wandb: ⭐️ View project at https://wandb.ai/seitomoyi-carnegie-mellon-university/HW3P2
wandb: 🚀 View run at https://wandb.ai/seitomoyi-carnegie-mellon-university/HW3P2/runs/zv2rmfw7


# Train Functions

In [21]:
def train_model(model, train_loader, criterion, optimizer):

    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    total_loss = 0

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.cuda.amp.autocast():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += loss.item()

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

        batch_bar.update() # Update tqdm bar

        # Another couple things you need for FP16.
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close() # You need this to close the tqdm bar

    return total_loss / len(train_loader)


def validate_model(model, val_loader, decoder, phoneme_map= LABELS):

    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    total_loss = 0
    vdist = 0

    for i, data in enumerate(val_loader):

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.inference_mode():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += float(loss)
        vdist += calculate_levenshtein(torch.permute(h, (1, 0, 2)), y, lh, ly, decoder, phoneme_map)

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), dist="{:.04f}".format(float(vdist / (i + 1))))

        batch_bar.update()

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close()
    total_loss = total_loss/len(val_loader)
    val_dist = vdist/len(val_loader)
    return total_loss, val_dist

## Training Setup

In [22]:
def save_model(model, optimizer, scheduler, metric, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         metric[0]                  : metric[1],
         'epoch'                    : epoch},
         path
    )

def load_model(path, model, metric= 'valid_acc', optimizer= None, scheduler= None):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    epoch   = checkpoint['epoch']
    metric  = checkpoint[metric]

    return [model, optimizer, scheduler, epoch, metric]

In [23]:
torch.cuda.empty_cache()
gc.collect()

for epoch in range(config['epochs']):

    print("\nEpoch: {}/{}".format(epoch+1, config['epochs']))
    curr_lr = optimizer.param_groups[0]['lr']
    train_loss = train_model(model, train_loader, criterion, optimizer)
    valid_loss, valid_dist = validate_model(model, val_loader, decoder, LABELS)
    scheduler.step()

    print("\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_loss, curr_lr))
    print("\tVal Dist {:.04f}\t Val Loss {:.04f}".format(valid_dist, valid_loss))
    
    wandb.log({
        'train_loss': train_loss,
        'valid_dist': valid_dist,
        'valid_loss': valid_loss,
        'lr'        : curr_lr
    })
    save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, epoch_model_path)
    #wandb.save(epoch_model_path)
    print("Saved epoch model")
    
    if valid_dist <= best_lev_dist:
        best_lev_dist = valid_dist
        save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, best_model_path)
        #wandb.save(best_model_path)
        print("Saved best model")

run.finish()


Epoch: 1/80


	Train Loss 3.5167	 Learning Rate 0.0020000
	Val Dist 71.8570	 Val Loss 3.3130
Saved epoch model
Saved best model

Epoch: 2/80


	Train Loss 3.3035	 Learning Rate 0.0019992
	Val Dist 58.2623	 Val Loss 3.2473
Saved epoch model
Saved best model

Epoch: 3/80


	Train Loss 2.3729	 Learning Rate 0.0019969
	Val Dist 39.9333	 Val Loss 1.7488
Saved epoch model
Saved best model

Epoch: 4/80


	Train Loss 1.4671	 Learning Rate 0.0019931
	Val Dist 25.2002	 Val Loss 1.1360
Saved epoch model
Saved best model

Epoch: 5/80


...

	Train Loss 0.1421	 Learning Rate 0.0000133
	Val Dist 5.0059	 Val Loss 0.2909
Saved epoch model

Epoch: 78/80


	Train Loss 0.1419	 Learning Rate 0.0000079
	Val Dist 5.0126	 Val Loss 0.2904
Saved epoch model

Epoch: 79/80


	Train Loss 0.1417	 Learning Rate 0.0000041
	Val Dist 5.0054	 Val Loss 0.2908
Saved epoch model

Epoch: 80/80


	Train Loss 0.1417	 Learning Rate 0.0000018
	Val Dist 5.0011	 Val Loss 0.2897
Saved epoch model


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:         lr ███████▇▇▇▆▆▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
wandb: train_loss █▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: valid_dist █▄▃▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: valid_loss ██▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: 
wandb: Run summary:
wandb:         lr 0.0
wandb: train_loss 0.14173
wandb: valid_dist 5.00109
wandb: valid_loss 0.2897
wandb: 
wandb: 🚀 View run scarlet-energy-21 at: https://wandb.ai/seitomoyi-carnegie-mellon-university/HW3P2/runs/zv2rmfw7
wandb: ⭐️ View project at: https://wandb.ai/seitomoyi-carnegie-mellon-university/HW3P2
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20241104_173225-zv2rmfw7/logs


# Generate Predictions and Submit to Kaggle

In [24]:
TEST_BEAM_WIDTH = config["beam_width"]

test_decoder    = cuda_ctc_decoder(tokens= LABELS, beam_size= TEST_BEAM_WIDTH)
results = []

checkpoint = torch.load(best_model_path)
model.load_state_dict(checkpoint['model_state_dict'])

model.eval()
print("Testing")
for data in tqdm(test_loader):

    x,_,lx,_= data
    x       = x.to(device)

    with torch.no_grad():
        h, lh = model(x, lx)

    prediction_string= decode_prediction(h, lh, decoder)
    results.extend(prediction_string)

    del x, lx, h, lh
    torch.cuda.empty_cache()

Testing


100%|██████████| 21/21 [00:13<00:00,  1.58it/s]


In [25]:
df = pd.DataFrame({'index': range(len(results)), 'label': results})
df.to_csv(csv_path, index=False)
print(df.head())

   index                                              label
0      0  -hihoptDErwUdbistIlfcrdInR-tRnApsAndkErAtsAndb...
1      1                    -stAfItIntuyuhIzbElikWnsAldhIm-
2      2  -@ftRRlinYtfclDAyElol@mpswUdlYtAphirAndDErDAsk...
3      3                       -hElWRbRtiAndhigUdInycrmYnd-
4      4      -nAmbR-dEn-frESnEliIzwetIGanyu-kUdnathAzbAnd-


In [26]:
# !kaggle competitions submit -c hw3p2-785-f24 -f submission.csv -m "I made it!"